# Pandera

Let's work through using `assert` to catch some data assumptions, then making it more solid with Pandera on the 30 minute resampled data - then you'll do some homework on the raw data.

----

In [1]:
import pandas as pd
import pandera as pa
from pandera.errors import SchemaErrors
import matplotlib.pyplot as plt

from utility import (
    make_max_water_by_temp_dataframe,
    set_common_mpl_styles,
    load_data,
    plot_day,
    make_estimated_water_plot,
    make_bin_edges,
    make_bootstrap,
)

FILENAME_KITCHEN = "Kitchen_export_202112311652.csv"

# Load kitchen data

In [2]:
df_raw = load_data(FILENAME_KITCHEN)

# now let's choose a day
day_to_choose = "2021-12-24"
print(f"Working on {day_to_choose}")
assert pd.to_datetime(day_to_choose) > df_raw.index.min(), "Must be in date range"
assert pd.to_datetime(day_to_choose) < df_raw.index.max(), "Must be in date range"

assert df_raw["t_c"].max() < 50, "Never expecting above 50C in the house"
assert (
    df_raw["rh"].min() > 0
), "Relative humidify can't be below 0% saturation in a normal house"

df_raw.head()

Date range 2021-12-22 17:00:00 - 2021-12-31 16:52:00
Working on 2021-12-24


,t_c,rh
timestamp,,
2021-12-22 17:00:00,14.0,48.1
2021-12-22 17:01:00,14.0,48.2
2021-12-22 17:02:00,14.0,48.1
2021-12-22 17:03:00,14.0,48.1
2021-12-22 17:04:00,14.0,48.2


In [3]:
df_30min = df_raw.resample("30min").mean()

df_30min.head()

,t_c,rh
timestamp,,
2021-12-22 17:00:00,14.916667,46.893333
2021-12-22 17:30:00,13.756667,48.800000
2021-12-22 18:00:00,13.860000,51.653333
2021-12-22 18:30:00,14.760000,53.906667
2021-12-22 19:00:00,15.053333,52.816667


# Try Pandera to catch errors with some debug info

In [4]:
# Examples to try
# rh must be >60 (not >0) - multiple examples, all on 1 line with lazy=True
# rh > 60 lazy=False to show 1 with a better output
# rh > 60 and rh < 70 lazy=True to show both sets 
# rh > 60 and rh < 70 lazy=False to show only the first

def sanity_check_data_with_index(df):
    min_date = "2021-01-01"
    max_date = "2022-12-12"
    schema = pa.DataFrameSchema(
        {
            "rh": pa.Column(float, checks=[pa.Check.gt(0), pa.Check.lt(100)]),
            #"rh": pa.Column(float, checks=[pa.Check.gt(60), pa.Check.lt(70)]),
        },
        index=pa.Index(
            "datetime64[ns]", checks=[pa.Check.gt(min_date), pa.Check.lt(max_date)]
        ),
        ordered=True,
    )
    validated_df = schema(df, lazy=True)
    
sanity_check_data_with_index(df_30min)

# Catch better descriptions

In [5]:
# Examples to try
# rh > 60 and rh < 70 lazy=True to show descriptions of errors

def sanity_check_data_with_index_exception(df):
    min_date = "2021-01-01"
    max_date = "2022-12-12"
    schema = pa.DataFrameSchema(
        {
            "rh": pa.Column(float, checks=[pa.Check.gt(0), pa.Check.lt(100)]),
            #"rh": pa.Column(float, checks=[pa.Check.gt(60), pa.Check.lt(70)]),
        },
        index=pa.Index(
            "datetime64[ns]", checks=[pa.Check.gt(min_date), pa.Check.lt(max_date)]
        ),
        ordered=True,
    )
    try:
        schema.validate(df, lazy=True)
    except SchemaErrors as err:
        print(err.failure_cases)  # dataframe of schema errors
        print(err.data)  # invalid dataframe
        
sanity_check_data_with_index_exception(df_30min)


# Homework

We've caught errors on the 30 minute resampled data - this is generated from `df_raw` where 30 * 1min datapoints are summarised (with `mean`) to make our processed data. What sins might exist in the raw data?

The function below will run on `df_raw`, not `df_30min`.

We've caught `rh` problems - now try to catch `t_c` problems. We'll expect a range of e.g. >10 to <35 Celcius in the house - can you test for that? Copy one of the above functions, give it a new name, then try it out. 

Can you update `rh` to check for a humidity between 30% and 100%?

Remember that `ordered=True` is set so if you defined the `t_c` row _after_ `rh` then it'll be out of order, you want `rh` and then `t_c` - but try to swap them so you can see the error you get.

What's the largest `min_date` you can get before you get an error?

Bonus - define a new `t_c_delta` column using:
```
    df = df.copy()
    df['t_c_delta'] = df['t_c'].shift() - df['t_c']
    df['t_c_delta'] = df['t_c_delta'].fillna(0)
```
which will give the 30min differences per rows, add a 3rd check for `t_c_delta` after `rh`, check for a sensible minimum and maximum change in temperature per 30 minute sampling gap. What's biggest change you might sensibly expect? That's a good starting point.

In [6]:
def sanity_check_raw(df):
    min_date = "2021-01-01"
    max_date = "2022-12-12"
    schema = pa.DataFrameSchema(
        {
            "rh": pa.Column(float, checks=[pa.Check.gt(00), pa.Check.lt(100)]),
        },
        index=pa.Index(
            "datetime64[ns]", checks=[pa.Check.gt(min_date), pa.Check.lt(max_date)]
        ),
        ordered=True,
    )
    validated_df = schema(df, lazy=True)

    try:
        schema.validate(df, lazy=True)
    except SchemaErrors as err:
        print(err.failure_cases)  # dataframe of schema errors
        print(err.data)  # invalid dataframe

# no need to reload the data, it is just convenient to have it to hand whilst you iterate here
df_raw = load_data(FILENAME_KITCHEN)
sanity_check_raw(df_raw)
print("No problems seen")

Date range 2021-12-22 17:00:00 - 2021-12-31 16:52:00
No problems seen
